In [25]:
import pyspark

spark = pyspark.sql.SparkSession \
    .builder \
    .appName("COVID-CXR META ETL") \
    .config("spark.some.config.option", "xxx") \
    .getOrCreate()

try:
    # once intialized, it's global in the jupyter notebook kernal; one context per notebook
    sc = pyspark.SparkContext.getOrCreate('local[*]')
except:
    sc = pyspark.SparkContext('local[*]')
sqlc = pyspark.SQLContext

In [72]:
INPUT_PATH_0_META

'./data/source/covid-chestxray-dataset-master/metadata.csv'

In [73]:
def etl_META_0():
    # src 0
    META_0 = spark.read.csv(INPUT_PATH_0_META, header=True, sep=",").rdd
    global _src0_url
    _src0_url = META_0.url.to_list()  # for duplicates in src 3
    META_0 = META_0[META_0.view.isin(["PA", "AP", "AP Supine", "AP semi erect", "AP erect"])]  # filter views
    META_0 = META_0[['patientid', 'filename', 'finding']]
    META_0.dropna(axis=0, how='any', inplace=True)
    META_0.rename(columns={'patientid': 'patient', 'filename': 'img', 'finding': 'label'}, inplace=True)
    META_0['src'] = 0
    META_0['label'] = META_0.label.apply(src0_label)
    META_0['img'] = META_0.img.apply(lambda path: os.path.join(INPUT_PATH_0_IMG, path))
    META_0 = META_0[META_0.label!='other']
    return META_0

In [75]:
df = spark.read.csv(INPUT_PATH_0_META, header=True, sep=",").rdd
print(df)
print(df.take(1))

MapPartitionsRDD[189] at javaToPython at <unknown>:0
[Row(patientid='2', offset='0', sex='M', age='65', finding='Pneumonia/Viral/COVID-19', RT_PCR_positive='Y', survival='Y', intubated='N', intubation_present='N', went_icu='N', in_icu='N', needed_supplemental_O2='Y', extubated=None, temperature=None, pO2_saturation=None, leukocyte_count=None, neutrophil_count=None, lymphocyte_count=None, view='PA', modality='X-ray', date='January 22, 2020', location='Cho Ray Hospital, Ho Chi Minh City, Vietnam', folder='images', filename='auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg', doi='10.1056/nejmc2001272', url='https://www.nejm.org/doi/full/10.1056/NEJMc2001272', license=None, clinical_notes='On January 22, 2020, a 65-year-old man with a history of hypertension, type 2 diabetes, coronary heart disease for which a stent had been implanted, and lung cancer was admitted to the emergency department of Cho Ray Hospital, the referral hospital in Ho Chi Minh City, for low-grade

In [35]:
spark

In [66]:
for i in dir(spark.read.format):
    if i[0] != '_':
        print(i)

In [19]:
sqlContext = sc.sqlContext

AttributeError: 'SparkContext' object has no attribute 'sqlContext'

In [ ]:
big_list = range(10000)
rdd = sc.parallelize(big_list, 2)
odds = rdd.filter(lambda x: x % 2 != 0)
odds.take(5)

In [18]:
rdd

NameError: name 'rdd' is not defined

In [64]:
"""
SELECT subject_id, sex, dob, ifnull(dod, "")
FROM PATIENT
"""

'\nSELECT subject_id, sex, dob, ifnull(dod, "")\nFROM PATIENT\n'

## meta

In [ ]:
from src.etl import *
from src.utils import *
import yaml
from collections import Counter
from collections import defaultdict

In [ ]:
params['data']['image_size']

In [ ]:
META = etl()

In [ ]:
print('label distribution: ', Counter(META.label), sep='\n')

## train / test split

In [ ]:
META = dataset_split(META)
META.head()

In [ ]:
print('label integer map: ', params['train']['labelmap'], sep='\n', end='\n\n')
for ds in (0, 1):
    print('{0} data from covid datasets: '.format({0: 'test', 1: 'train'}[ds]), Counter(META[META.train==ds].label), sep='\n', end='\n\n')

In [ ]:
# cache image meta
if not os.path.isdir(SAVE_PATH):
    os.makedirs(SAVE_PATH)
with open(os.path.join(SAVE_PATH, 'meta'), 'wb') as pickle_file:
    pickle.dump(META, pickle_file, pickle.HIGHEST_PROTOCOL)
    print(f"META data saved at {os.path.join(SAVE_PATH, 'meta')}")

In [ ]:
%%time
if False:
    # -------------------------------------------------------------------------------------------------------------------
    # imagge file to matrix  
    # NOTE: this needs model to load all data at once into cache which take up too much storage, 
    #       making computing inefficient, NOT GOOD
    # -------------------------------------------------------------------------------------------------------------------
    # train_data / test_data structure: 
    #                        {'covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer},
    #                        '!covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer}}
    TRAIN_DATA, TEST_DATA = datafromfile(META)
    # a sanity check
    print(f"train covid sample #: {len(TRAIN_DATA['covid']['label'])}")
    print(f"train !covid sample #: {len(TRAIN_DATA['!covid']['label'])}")
    print(f"test  covid sample #: {len(TEST_DATA['covid']['label'])}")
    print(f"test  !covid sample #: {len(TEST_DATA['!covid']['label'])}")
    # cache image data
    with open(os.path.join(SAVE_PATH, 'train.data'), 'wb') as pickle_file:
        pickle.dump(TRAIN_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'train.data')}")
    with open(os.path.join(SAVE_PATH, 'test.data'), 'wb') as pickle_file:
        pickle.dump(TEST_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'test.data')}")
else:
    # merge imgaes together instead, and transform image to batches of matrix during training
    mergesoure(META)

In [ ]:
# sanity check for image shape, they need to consistent with model input
import cv2
import glob
import numpy as np
for i in np.random.choice(META.index, 10):
    print(i, cv2.imread(META.loc[i, 'imgid']).shape)